In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import os
import datetime
import matplotlib.pyplot as plt
import contextily as cx

### 1. 读取全部数据

In [2]:
df_raw = pd.read_csv('sample 10000.csv', usecols= [0,2,3,5, 28,29, 31,32, 33, 34, 35 ])
name_list = df_raw.columns.tolist()

In [3]:
r = pd.read_csv('000000_0_fb744a52-2d44-4e8d-8ff1-85aeda22a123_20231207_153237_00071_9h6es', 
                usecols= [0,2,3,5, 28,29, 31,32, 33, 34, 35 ],
                on_bad_lines='warn',header=None, names=name_list, iterator=True)

In [4]:
# df_test = pd.read_csv('000000_0_fb744a52-2d44-4e8d-8ff1-85aeda22a123_20231207_153237_00071_9h6es', on_bad_lines='warn', nrows=200000)

In [4]:
loop = True
chunkSize = 100000
chunks = []
while loop:
    try:
        chunk = r.get_chunk(chunkSize)
        chunks.append(chunk)
     #停止迭代，数据读到尾部，结束循环。
    except StopIteration:
        loop = False
        print("文件读取完毕.")
df = pd.concat(chunks, ignore_index=True)

文件读取完毕.


In [5]:
df['date'] = pd.to_datetime(df["timestamp"], unit='s') + pd.Timedelta(days=8/24)  #pd.to_datetime()得到的是utc时间，utc+8小时=本地时间

In [6]:
df[df['price_sqft_home']=='commercia']

,device_id,latitude,longitude,timestamp,device_home_latitude,device_home_longitude,device_work_latitude,device_work_longitude,location_type,price_sqft_home,price_sqft_location,date
166436,f34b8f61-40b0-4a1f-853a-ebb71566a6eb,22.336737,114.149350,1694344031,22.336130172093025,114.1494757023256,22.3356154090164,114.14922370245901,building-industrial,commercia,18568.0,2023-09-10 19:07:11
181054,f34b8f61-40b0-4a1f-853a-ebb71566a6eb,22.336704,114.149374,1694310338,22.336130172093025,114.1494757023256,22.3356154090164,114.14922370245901,building-industrial,commercia,18568.0,2023-09-10 09:45:38
241981,304d8a3a-8029-44cd-8b7d-4dc6e9e42637,22.336991,114.149661,1694050124,22.433829526315794,113.99132073684211,22.33686125,114.14962560714285,building-industrial,commercia,\N,2023-09-07 09:28:44
241982,304d8a3a-8029-44cd-8b7d-4dc6e9e42637,22.337086,114.149612,1694053673,22.433829526315794,113.99132073684211,22.33686125,114.14962560714285,building-industrial,commercia,\N,2023-09-07 10:27:53
241983,304d8a3a-8029-44cd-8b7d-4dc6e9e42637,22.336941,114.149661,1694076239,22.433829526315794,113.99132073684211,22.33686125,114.14962560714285,building-industrial,commercia,\N,2023-09-07 16:43:59
...,...,...,...,...,...,...,...,...,...,...,...,...
61401068,67dc5b08-f3a4-4196-9b27-67157adc2c65,22.336720,114.149376,1694308044,22.34194308241758,114.15555776520146,22.243349858974366,114.1489385410256,building-industrial,commercia,\N,2023-09-10 09:07:24
61401805,5877dc00-fd25-4f22-bb5c-f70d13b8fbd6,22.336600,114.149642,1693893175,22.36375270786517,114.12952645056181,22.36364442556391,114.12954632706767,building-industrial,commercia,\N,2023-09-05 13:52:55
61558557,c1d32bd2-8e76-4cad-a301-73ef8e3544b9,22.336720,114.149376,1694603008,22.462906899999997,114.18987569999993,22.457788199999996,114.17191110000002,building-industrial,commercia,\N,2023-09-13 19:03:28
61699636,fb2e6651-7aa2-4bb1-8f89-be8aaa99a61a,22.336720,114.149376,1694243791,22.371861462068964,114.11984651149426,22.365761935294117,114.12035921764706,building-industrial,commercia,14613.42857142857,2023-09-09 15:16:31


In [6]:
## 删除df[df['price_sqft_home']=='commercia']的行
df = df.drop(df[df['price_sqft_home']=='commercia'].index)
df

,device_id,latitude,longitude,timestamp,device_home_latitude,device_home_longitude,device_work_latitude,device_work_longitude,location_type,price_sqft_home,price_sqft_location,date
0,f654d4a6-3850-4021-98df-1e4de5d5a945,22.291246,114.198296,1693528861,22.37802089590164,114.2580357467213,22.378009384552843,114.25804358699189,\N,\N,14695.6,2023-09-01 08:41:01
1,f654d4a6-3850-4021-98df-1e4de5d5a945,22.269249,114.240915,1693553045,22.37802089590164,114.2580357467213,22.378009384552843,114.25804358699189,building-garage,\N,\N,2023-09-01 15:24:05
2,f6580b80-1451-48b9-b99f-685c17a54fc8,22.339961,114.134779,1693738855,22.36587730666667,114.13682803666667,22.365859175409835,114.13682745573773,building-hospital,13322.4,\N,2023-09-03 19:00:55
3,f6580b80-1451-48b9-b99f-685c17a54fc8,22.339928,114.134885,1693739241,22.36587730666667,114.13682803666667,22.365859175409835,114.13682745573773,building-hospital,13322.4,\N,2023-09-03 19:07:21
4,f6580b80-1451-48b9-b99f-685c17a54fc8,22.339945,114.134766,1693739590,22.36587730666667,114.13682803666667,22.365859175409835,114.13682745573773,building-hospital,13322.4,\N,2023-09-03 19:13:10
...,...,...,...,...,...,...,...,...,...,...,...,...
62969747,b459657d-9ab1-4f73-9154-28f6044e2df7,22.321577,114.260256,1694439893,22.321534486486485,114.2602551189189,22.321736599999994,114.26014184444443,\N,13971.285714285714,13971.285714285714,2023-09-11 21:44:53
62969748,b460a842-0630-4ebf-963b-6285b81f5574,22.310000,114.230000,1694668126,\N,\N,\N,\N,\N,\N,\N,2023-09-14 13:08:46
62969749,b4797889-7ba8-6302-2099-7fd7d0cf6e40,22.281597,114.159020,1693716923,22.266985415669517,114.23429762393165,22.267076286445782,114.23430978042168,building-commercial,\N,\N,2023-09-03 12:55:23
62969750,b4797889-7ba8-6302-2099-7fd7d0cf6e40,22.284408,114.143100,1694339102,22.266985415669517,114.23429762393165,22.267076286445782,114.23430978042168,\N,\N,24369.0,2023-09-10 17:45:02


In [11]:
np.size(df['device_id'].unique())

370029

In [8]:
df.to_csv('dataset_all.csv', index=False) 

In [10]:
df.dtypes

device_id                        object
latitude                        float64
longitude                       float64
timestamp                         int64
device_home_latitude             object
device_home_longitude            object
device_work_latitude             object
device_work_longitude            object
location_type                    object
price_sqft_home                  object
price_sqft_location              object
date                     datetime64[ns]
dtype: object

### 2. 数据准备：
#### dataset1: building_info:以building为粒度，储存每个building的价格和type，需要全部数据集（62968903 rows × 4 columns）
#### dataset2: 有房间信息的，获得这部分id的房间位置（59839138 rows × 5 columns）
#### dataset3: 没有房间信息，且1点到3点有数据的，用于确定这部分id的房间位置(164381 rows × 3 columns) 
#### dataset4: 有房间信息，且1点到3点有数据的，用于判断1点到3点的位置是否可以被认为是房间位置。（4149501 rows × 5 columns）

In [18]:
## dataset 1
df1 = df[['latitude','longitude','location_type','price_sqft_location']]
# df1.to_csv('dataset_1.csv', index=False) 
df1

,latitude,longitude,location_type,price_sqft_location
0,22.291246,114.198296,\N,14695.6
1,22.269249,114.240915,building-garage,\N
2,22.339961,114.134779,building-hospital,\N
3,22.339928,114.134885,building-hospital,\N
4,22.339945,114.134766,building-hospital,\N
...,...,...,...,...
62969747,22.321577,114.260256,\N,13971.285714285714
62969748,22.310000,114.230000,\N,\N
62969749,22.281597,114.159020,building-commercial,\N
62969750,22.284408,114.143100,\N,24369.0


In [20]:
df1[df1['price_sqft_location'] == r'\N']

,latitude,longitude,location_type,price_sqft_location
1,22.269249,114.240915,building-garage,\N
2,22.339961,114.134779,building-hospital,\N
3,22.339928,114.134885,building-hospital,\N
4,22.339945,114.134766,building-hospital,\N
5,22.339966,114.134692,building-hospital,\N
...,...,...,...,...
62969737,22.397985,114.210910,\N,\N
62969738,22.416069,114.229677,building-school,\N
62969748,22.310000,114.230000,\N,\N
62969749,22.281597,114.159020,building-commercial,\N


In [10]:
df1 = df1[df1['price_sqft_location'] != r'\N']
df1

,latitude,longitude,location_type,price_sqft_location
0,22.291246,114.198296,\N,14695.6
11,22.342749,114.183385,building-apartments,15450.666666666666
12,22.342748,114.183376,building-apartments,15450.666666666666
13,22.342758,114.183382,building-apartments,15450.666666666666
14,22.372702,114.178356,\N,23536.636363636364
...,...,...,...,...
62969744,22.321530,114.260220,\N,13971.285714285714
62969745,22.321531,114.260227,\N,13971.285714285714
62969746,22.321526,114.260260,\N,13971.285714285714
62969747,22.321577,114.260256,\N,13971.285714285714


In [13]:
np.size(df1['location_type'].unique())

125

In [14]:
df1_1 = df1[df1['location_type'] != r'\N']
np.size(df1_1['location_type'].unique())

124

In [7]:
df2 = df[df['device_home_latitude'] != r'\N']
df2 = df2[['device_id','latitude','longitude','device_home_latitude','device_home_longitude','date']]

In [8]:
df2.to_csv('dataset_4_raw.csv', index=False) 

In [16]:
## dataset4
df4 = df2[(df2['date'].dt.hour >= 1) & (df2['date'].dt.hour <= 3)] # 凌晨1：00至3：59的数据
df4 = df4[['device_id','latitude','longitude','device_home_latitude','device_home_longitude']]
df4
# df4.to_csv('dataset_4.csv', index=False) 

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_7804\2707428687.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df4 = df2[(df2['date'].dt.hour >= 1) & (df['date'].dt.hour <= 3)] # 凌晨1：00至3：59的数据


,device_id,latitude,longitude,device_home_latitude,device_home_longitude
31,f46a65f1-db93-477b-a2e6-798f09b778a3,22.326504,114.257659,22.327950066666666,114.25898536190475
32,f46a65f1-db93-477b-a2e6-798f09b778a3,22.326499,114.257590,22.327950066666666,114.25898536190475
163,f6ec9f4b-8020-467c-be2c-df948aebe5dc,22.441663,114.072847,22.441665772670806,114.07283580062109
164,f6ec9f4b-8020-467c-be2c-df948aebe5dc,22.441631,114.072864,22.441665772670806,114.07283580062109
165,f6ec9f4b-8020-467c-be2c-df948aebe5dc,22.441676,114.072764,22.441665772670806,114.07283580062109
...,...,...,...,...,...
62969679,b56a2980-91b2-415d-945f-8a445d5bbe60,22.316356,114.174776,22.316360711564624,114.17478383129252
62969680,b56a2980-91b2-415d-945f-8a445d5bbe60,22.316363,114.174783,22.316360711564624,114.17478383129252
62969681,b56a2980-91b2-415d-945f-8a445d5bbe60,22.316363,114.174786,22.316360711564624,114.17478383129252
62969682,b56a2980-91b2-415d-945f-8a445d5bbe60,22.316358,114.174779,22.316360711564624,114.17478383129252


In [11]:
np.size(df2['device_id'].unique())

105037

In [17]:
np.size(df4['device_id'].unique())

91096

In [9]:
## dataset 2
df2_1 = df2[['device_id','latitude','longitude','device_home_latitude','device_home_longitude']]
df2_1
# df2_1.to_csv('dataset_2.csv', index=False) 

,device_id,latitude,longitude,device_home_latitude,device_home_longitude
0,f654d4a6-3850-4021-98df-1e4de5d5a945,22.291246,114.198296,22.37802089590164,114.2580357467213
1,f654d4a6-3850-4021-98df-1e4de5d5a945,22.269249,114.240915,22.37802089590164,114.2580357467213
2,f6580b80-1451-48b9-b99f-685c17a54fc8,22.339961,114.134779,22.36587730666667,114.13682803666667
3,f6580b80-1451-48b9-b99f-685c17a54fc8,22.339928,114.134885,22.36587730666667,114.13682803666667
4,f6580b80-1451-48b9-b99f-685c17a54fc8,22.339945,114.134766,22.36587730666667,114.13682803666667
...,...,...,...,...,...
62969745,b459657d-9ab1-4f73-9154-28f6044e2df7,22.321531,114.260227,22.321534486486485,114.2602551189189
62969746,b459657d-9ab1-4f73-9154-28f6044e2df7,22.321526,114.260260,22.321534486486485,114.2602551189189
62969747,b459657d-9ab1-4f73-9154-28f6044e2df7,22.321577,114.260256,22.321534486486485,114.2602551189189
62969749,b4797889-7ba8-6302-2099-7fd7d0cf6e40,22.281597,114.159020,22.266985415669517,114.23429762393165


In [10]:
np.size(df2_1['device_id'].unique())

105037

In [14]:
# df2_1.to_csv('dataset_2.csv', index=False) 

In [11]:
df3_raw = df[df['device_home_latitude'] == r'\N']
df3_raw = df3_raw[['device_id','latitude','longitude','date']]
df3_raw

,device_id,latitude,longitude,date
10,f664b4d6-5809-413e-b49b-3605d0515503,22.338962,114.207170,2023-09-04 07:41:03
85,e9017f7c-c52f-4a6a-926e-b1c0976b1248,22.288587,114.195222,2023-09-05 11:16:02
130,f6a9d8b6-5e2c-488f-9ee7-f58e97e5d5de,22.340000,114.200000,2023-09-14 11:21:56
235,ff22c280-126f-600f-214c-0ab0411394db,22.370519,114.132008,2023-09-01 18:15:22
236,ff22c280-126f-600f-214c-0ab0411394db,22.368826,114.118049,2023-09-01 20:50:06
...,...,...,...,...
62969730,b58ded3e-5a73-47e9-8af1-58ba760ac6d0,22.395124,113.974522,2023-09-06 06:47:37
62969731,b58ded3e-5a73-47e9-8af1-58ba760ac6d0,22.395138,113.974516,2023-09-06 07:15:54
62969732,b5933b4e-0e52-4398-8253-1f27bb4bec07,22.273868,114.187822,2023-09-15 03:06:44
62969748,b460a842-0630-4ebf-963b-6285b81f5574,22.310000,114.230000,2023-09-14 13:08:46


In [12]:
np.size(df3_raw['device_id'].unique())

264992

In [13]:
# df3_raw.to_csv('dataset_3_raw.csv', index=False) 

In [14]:
df3 = df3_raw[(df3_raw['date'].dt.hour >= 1) & (df3_raw['date'].dt.hour <= 3)] # 凌晨1：00至3：59的数据
df3

,device_id,latitude,longitude,date
1238,e60fbeb4-7e78-4985-bcc3-09ff874cb7e3,22.445384,114.035978,2023-09-07 01:04:22
1239,e60fbeb4-7e78-4985-bcc3-09ff874cb7e3,22.445352,114.035938,2023-09-07 01:05:11
1933,ec11d950-75e3-4d22-b27f-4e7203f94342,22.364642,114.136940,2023-09-04 02:34:01
2461,96c515c6-540f-45d3-9c9b-ff4edb701896,22.384730,114.196530,2023-09-02 03:44:40
2462,96e87f08-a206-4ba9-aaed-395d6495aecc,22.365936,114.057621,2023-09-15 01:18:35
...,...,...,...,...
62968933,931adaea-055c-4203-bba9-8c08786277cc,22.501801,114.129160,2023-09-02 02:19:45
62969333,afc663ee-9b82-4bfc-a45f-6cce84cb35b2,22.424879,114.229265,2023-09-04 03:41:49
62969334,afc663ee-9b82-4bfc-a45f-6cce84cb35b2,22.424917,114.229258,2023-09-04 03:47:44
62969335,afc663ee-9b82-4bfc-a45f-6cce84cb35b2,22.424910,114.229247,2023-09-04 03:58:45


In [15]:
np.size(df3['device_id'].unique())

31126

In [ ]:
####################################################################################

In [16]:
# # Reduce memory usage

# data = df_cleaned.drop(columns=['latitude','longitude','timestamp','device_home_longitude','device_home_latitude','device_work_latitude','device_work_longitude','location_type','price_sqft_location','date'])

# data_int = data.select_dtypes(include=['int'])
# converted_int = data_int.apply(pd.to_numeric,downcast='unsigned')

# data_float = data.select_dtypes(include=['float'])
# converted_float = data_float.apply(pd.to_numeric,downcast='float')

# optimized_data = data.copy()
# optimized_data[converted_int.columns] = converted_int
# optimized_data[converted_float.columns] = converted_float

# data_obj = optimized_data.select_dtypes(include=['object']).copy()
# device_id_cate = data_obj.device_id.astype('category')
# optimized_data['device_id'] = device_id_cate